# Homework 1 ATED - Group 1

In [1]:
%pylab inline
import math
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

Populating the interactive namespace from numpy and matplotlib


## HW1-1

#### You are short 100 at-the-money call option contracts on the S&P 500 index expiring in one year with a contract multiplier of 100. The current index level is 2600, the interest and dividend rates are zero.

##### Simulate the evolution of the index level at periods Δt as a geometric Brownian motion with volatility $\sigma$ (free parameter) and calculate the corresponding call value, delta, gamma and theta using a fixed 20\% implied volatility.

In [2]:
# CODE

##### Then simulate your actual cumulative P\&L when periodically delta-hedging your position (assuming you can trade the index as an asset) and compare it against the proxy formula on slide 11 for the following matrix of parameters.

In [2]:
# CODE

##### Provide a statistical analysis of your results over 10,000 simulations, where one simulation is an entire index path.

In [2]:
# CODE

## HW1-3

#### On March 29, 2018 the S\&P 500 index (SPX) is 2611.53 and the 12-month VIX is 21.28. The implied volatility smile of SPX options expiring on March 15, 2019 is given as:
$$ \sigma^* = \sqrt{a + b \left( \rho (x - m)  + \sqrt{(x-m)^2 + s^2} \right)} $$
#### where $a = 0.009$, $b = 0.11$, $\rho = −0.12$, $m = 0.2$, $s = 0.05$, $x = \log \frac{K}{F}$ is log-moneyness and $F = 2625.10$ is the forward price. The continuous interest rate is 2.09\% p.a.

##### Draw the implied volatility smile curve for strikes $500 \leq K \leq 5000$.

In [2]:
# CODE

##### Calculate the fair strike K var of a variance swap expiring on March 15, 2019 with the method of your choice. How close is your calculation to the 12-month VIX? Why is it not exactly the same?

In [ ]:
# CODE